In [ ]:
#importer les bibliothèques

In [ ]:
import json
import pandas as pd
import datetime

from tabula import read_pdf
import tabula
from tabulate import tabulate
import os
import copy

In [ ]:
#télécharger les fichiers JSON des villes & regions

In [ ]:
!wget https://raw.githubusercontent.com/alaouy/sql-moroccan-cities/master/json/ville.json

In [ ]:
!wget https://raw.githubusercontent.com/alaouy/sql-moroccan-cities/master/json/region.json

In [ ]:
#construite un pandas data frame pour les villes et les régions

In [ ]:
cities = pd.read_json("ville.json",orient="records")
cities.rename(columns={'id': 'id_ville', 'region': 'id_region'}, inplace=True)

In [ ]:
regions = pd.read_json("region.json",orient="records")
regions.rename(columns={'id': 'id_region'}, inplace=True)

In [ ]:
cities_regions = pd.merge(cities,regions)
cities_regions.head(2)

In [ ]:
#sur Linux WSL / google collab test du téléchargement pour un seul fichier

In [ ]:
!wget http://81.192.10.228/wp-content/uploads/2022/03/1_3_2022.pdf

In [ ]:
#parametrage du téléchargement pour plusieurs fichiers sur une marge de date

In [ ]:
start = datetime.datetime(2021,3,1)
end = datetime.datetime(2022,3,1)

In [ ]:
#téléchargement de tout les fichiers vers un dossier in/

In [ ]:
d = start
c=1
while(d<end):
    try:
        print(d.year,d.month,d.day)
        month = str(d.month)
        if d.month<10:
            month = "0"+month
        os.system(f"wget http://81.192.10.228/wp-content/uploads/{d.year}/{month}/{d.day}_{d.month}_{d.year}.pdf --directory-prefix=in/")
#         print(f"wget http://81.192.10.228/wp-content/uploads/{d.year}/{month}/{d.day}_{d.month}_{d.year}.pdf")
        d = d+datetime.timedelta(days=7)
    except Exception as e:
        print("\n-> ERROR!!!!",e)
    print("-----------")

In [ ]:
#lister tout les fichiers téléchargés

In [ ]:
import os
import re
filepath="./in/" #root path for checking
full_path = filepath
filepattern=r"\b(\w*.pdf)"
filepattern = full_path.replace("/","\/")+filepattern
file_pattern = re.compile(filepattern)

print("Searching...")
counter=0
tables=[]
try:
  #######################################
    for dirpath, subdirs, files in os.walk(full_path):
        for x in files:
            if x.endswith(".pdf"):        
                if re.match(file_pattern, os.path.join(dirpath, x)):
                    data = os.path.join(dirpath, x).split("/")[-1].split(".")[0]
                    tables.append(data.upper())
                    counter+=1
#                     print("")
#                     context['task_instance'].xcom_push('file_name', files)
#                     return True
    print(f"ALL DONE! {counter} sources/tables found")
#     return False
except:
    print("OOOOPS ERORRS ARE HERE!")
#     return False
# print("\n\n",tables)

In [ ]:
# Conversion de tout les fichiers téléchargés vers des csv non nettoyés

In [ ]:

for file in tables:
    try:
        print(file+".pdf")        
        tabula.convert_into(f"./in/{file}.pdf",f"./out/{file}.csv",output_format="csv",pages="all")

    except Exception as e:
        print("\n-> ERROR!!!!",e)
    print("-----------")


In [ ]:
#dev code abandonné

In [ ]:

#tabula.convert_into("1_3_2022.pdf","output.csv",output_format="csv",pages="all")

# import camelot
 
# # extract all the tables in the PDF file
# abc = camelot.read_pdf("1_3_2022.pdf")   #address of file location
 
# # print the first table as Pandas DataFrame
# print(abc[0].df)

In [ ]:
# from csv import reader
# # open file in read mode
# with open('./out/2_11_2021.csv', 'r') as read_obj:
#     # pass the file object to reader() to get the reader object
#     csv_reader = reader(read_obj)
#     # Iterate over each row in the csv using reader object
#     all_rows = []
#     for row in csv_reader:
#         # row variable is a list that represents a row in csv
#         all_rows.append(row)
# data = all_rows[4:]
# columns = ["BARRAGES","CAPACITE NORMALE (Mm3)","RESERVE (Mm3)","TAUX DE REMPLISSAGE ( % )","RESERVE (Mm3) Last Year","TAUX DE REMPLISSAGE ( % ) Last Year"]
# df = pd.DataFrame(data=data,columns=columns)
# df["date"] =  datetime.datetime.strptime("2_11_2021", '%d_%m_%Y')
# df

In [ ]:
#lecture de tout les csv en sorti avec la lib csv et la concaténation de toute la data dans un seul csv

In [ ]:
dfs = []
for file in tables:
    from csv import reader
    # open file in read mode
    with open(f'./out/{file}.csv', 'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        all_rows = []
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            all_rows.append(row)
    data = all_rows[4:]
    columns = ["barrages","capacite_normal_mm3","reserve_mm3","taux_de_remplissage_percent","reserve_mm3_last_year","taux_de_remplissage_percent_last_year"]
    df = pd.DataFrame(data=data,columns=columns)
    df["date"] =  datetime.datetime.strptime(file, '%d_%m_%Y').strftime("%Y-%m-%d")
    dfs.append(copy.copy(df))

In [ ]:
count=0
for i in range(len(dfs)):
    count+=len(dfs[i])
print(count)

In [ ]:
all_data = pd.concat(dfs)
all_data = all_data.reset_index()
all_data = all_data.drop(["index"],axis=1)
all_data

In [ ]:
all_data["barrages"].unique()

In [ ]:
#choisir vos barrages (le plus proches de votre ville natale )

In [ ]:
selected_barages = ["MANSOUR EDDAHBI","TANGER - MEDITERRANEE"]

In [ ]:
selected_data = all_data[all_data["barrages"].isin(selected_barages)]
selected_data.reset_index(drop=True, inplace=True)
selected_data["ville"] = ""
selected_data["region"] = ""
selected_data.loc[selected_data['barrages'] == 'TANGER - MEDITERRANEE', 'ville'] = "TANGER"
selected_data.loc[selected_data['barrages'] == 'TANGER - MEDITERRANEE', 'region'] = "Tanger-Tetouan-Al Hoceima"

selected_data.loc[selected_data['barrages'] == 'MANSOUR EDDAHBI', 'ville'] = "Ouarzazate"
selected_data.loc[selected_data['barrages'] == 'MANSOUR EDDAHBI', 'region'] = "Draa-Tafilalet"

In [ ]:
selected_data

In [ ]:
#souvegarde de votre data

In [ ]:
selected_data.to_csv("./barrage_data.csv",index=False,encoding="UTF-8")